In [1]:
import alist_loader
import numpy as np
import jax
import jax.numpy as jnp

import matplotlib.pyplot as plt
import japanize_matplotlib
from jax.example_libraries import optimizers
from functools import partial

In [2]:
filename = "../../DU-Book/Chapter_5/BCH_31_16_3_strip.alist"

In [3]:
noise_std = 0.5
K = 100
bp_itr = 5

In [7]:
H, U, V = alist_loader.load_alist(filename)
H = jnp.array(H)
U = jnp.array(U)
V = jnp.array(V)

n = 31, m = 15
cmap = 7, rmap = 8


In [52]:
n = H.shape[1]
m = H.shape[0]

In [8]:
def bmod(x):
    return x - 2*jnp.floor(x/2)

In [28]:
@jax.jit
def parity_check(x):
    print(jnp.sum(H@x))
    tmp = jnp.where(jnp.sum(bmod(H@x)) > 0, 1, 0)
    return tmp
               

In [59]:
def random_codeword():
    x = jnp.array(np.random.randint(0, 2, n))
    for i in range(1, m+1):
        k = m - i + 1
        parity = 0
        for j in range(k+1, n+1):
            parity += H[k-1,j-1] * x[j-1]
        x = x.at[k-1].set(parity % 2)
    p = parity_check(x)
    if p == 0:
        return x
    else:
        print("encoding error!")
    return x

In [60]:
bmod(7)

Array(1., dtype=float32, weak_type=True)

In [61]:
7%2

1

In [62]:
x = random_codeword()
parity_check(x)

Traced<ShapedArray(int32[])>with<DynamicJaxprTrace(level=1/0)>


Array(0, dtype=int32, weak_type=True)

In [71]:
def mini_batch(K):
    x = np.zeros((n, K))
    y = np.zeros((n, K))
    for j in range(1, K+1):
        x[:, j-1] = -2*jax.device_get(random_codeword()) + 1
        y[:, j-1] = x[:, j-1] + noise_std*np.random.randn(n)
    return jnp.array(x), jnp.array(y)